In [1]:
import os
import gzip
import json
import h3
import pandas as pd
import numpy as np
from shapely.geometry import shape, Point

# --- 1. Path Configuration ---
home_dir = os.path.expanduser("~")
# Input folder: Contains 10 city .geojson.gz boundary files
input_dir = os.path.join(home_dir, "Desktop", "Infor 301", "final", "row data")
# Output folder: Where the resulting H3 task list CSVs will be stored
output_dir = os.path.join(home_dir, "Desktop", "Infor 301", "final", "task_lists")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📂 Created output directory: {output_dir}")

def generate_city_h3_grid_res8(file_path, city_name, resolution=8):
    """
    Reads a compressed GeoJSON boundary and converts it into a list of H3 Res 8 indices.
    """
    print(f"🔄 Processing city: {city_name} (Resolution: {resolution}) ...")
    
    # 1. Load boundary data (handling .gz compressed format)
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # 2. Extract geometry (supports various OSM-Boundary structures)
    if 'features' in raw_data and len(raw_data['features']) > 0:
        city_geometry = shape(raw_data['features'][0]['geometry'])
    else:
        city_geometry = shape(raw_data['geometry'])
    
    # 3. Get the Bounding Box of the city
    min_lng, min_lat, max_lng, max_lat = city_geometry.bounds
    
    # 4. Optimized Sampling Strategy for Res 8
    # H3 Res 8 edge length is ~461m.
    # To ensure coverage while maintaining speed, step is set to ~330m (0.003 degrees).
    step = 0.003 
    
    lng_range = np.arange(min_lng, max_lng, step)
    lat_range = np.arange(min_lat, max_lat, step)
    
    potential_hexes = set()
    
    # 5. Core Logic: Check if sampling points are inside the city boundary
    for lng in lng_range:
        for lat in lat_range:
            point = Point(lng, lat)
            if city_geometry.contains(point):
                try:
                    # Compatibility for H3 v4.x
                    hex_id = h3.latlng_to_cell(lat, lng, resolution)
                except AttributeError:
                    # Compatibility for H3 v3.x
                    hex_id = h3.geo_to_h3(lat, lng, resolution)
                potential_hexes.add(hex_id)
    
    # 6. Convert hex indices to list with coordinate centers
    nodes_data = []
    for h_index in potential_hexes:
        try:
            c_lat, c_lng = h3.cell_to_latlng(h_index)
        except AttributeError:
            c_lat, c_lng = h3.h3_to_geo(h_index)
        
        nodes_data.append({
            "h3_index": h_index, 
            "latitude": c_lat, 
            "longitude": c_lng,
            "city_name": city_name
        })
    
    # 7. Export to CSV for POI scraping and 3D visualization
    df = pd.DataFrame(nodes_data)
    output_filename = f"{city_name}_Res8_task_list.csv"
    output_path = os.path.join(output_dir, output_filename)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    return len(df)

# --- 2. Automation Logic ---
if __name__ == "__main__":
    summary_report = []
    
    # Set target resolution to 8 (approx 0.74 sq km)
    target_resolution = 8
    
    if not os.path.exists(input_dir):
        print(f"❌ Input path not found: {input_dir}")
    else:
        # Detect all .geojson.gz files in the folder
        target_files = [f for f in os.listdir(input_dir) if f.endswith(".geojson.gz")]
        print(f"🔍 Found {len(target_files)} boundary files to process.")
        
        for file_name in target_files:
            city_key = file_name.replace(".geojson.gz", "").split('.')[0]
            try:
                hex_count = generate_city_h3_grid_res8(os.path.join(input_dir, file_name), city_key, resolution=target_resolution)
                print(f"✅ {city_key}: Successfully generated {hex_count} H3 cells.")
                summary_report.append({"City": city_key, "Hex_Count": hex_count})
            except Exception as e:
                print(f"❌ Error processing {city_key}: {str(e)}")

        # 8. Final Statistical Summary
        if summary_report:
            print("\n" + "="*60)
            print("📊 TRANSATLANTIC 10-CITY H3 (RES 8) SUMMARY")
            print("-" * 60)
            df_summary = pd.DataFrame(summary_report)
            # Area estimation: H3 Res 8 average area is ~0.737 sq km
            df_summary['Estimated_Area_km2'] = (df_summary['Hex_Count'] * 0.737).round(2)
            print(df_summary.to_string(index=False))
            print("="*60)
            print(f"🚀 All task lists saved to: {output_dir}")

📂 Created output directory: /Users/oushilin/Desktop/Infor 301/final/task_lists
🔍 Found 10 boundary files to process.
🔄 Processing city: Glasgow City (Resolution: 8) ...
✅ Glasgow City: Successfully generated 321 H3 cells.
🔄 Processing city: West Midlands (Resolution: 8) ...
✅ West Midlands: Successfully generated 1465 H3 cells.
🔄 Processing city: Los Angeles (Resolution: 8) ...
✅ Los Angeles: Successfully generated 1860 H3 cells.
🔄 Processing city: Houston (Resolution: 8) ...
✅ Houston: Successfully generated 2220 H3 cells.
🔄 Processing city: Greater Manchester (Resolution: 8) ...
✅ Greater Manchester: Successfully generated 2046 H3 cells.
🔄 Processing city: NewYork (Resolution: 8) ...
✅ NewYork: Successfully generated 1739 H3 cells.
🔄 Processing city: San Francisco (Resolution: 8) ...
✅ San Francisco: Successfully generated 869 H3 cells.
🔄 Processing city: West Yorkshire (Resolution: 8) ...
✅ West Yorkshire: Successfully generated 3246 H3 cells.
🔄 Processing city: Chicago (Resolution: